# Sistem Rekomendasi Film Menggunakan Content-Based dan Collaborative Filtering

## Project Overview
Sistem rekomendasi film bertujuan membantu pengguna menemukan film yang sesuai dengan preferensi mereka secara otomatis. Dengan banyaknya pilihan film, sistem ini dapat meningkatkan pengalaman pengguna dan membantu platform streaming meningkatkan engagement.

## Business Understanding
Permasalahan: Pengguna sering kesulitan memilih film yang sesuai dengan selera mereka di antara ribuan pilihan.
Solusi: Membangun sistem rekomendasi yang dapat memberikan saran film secara personal menggunakan dua pendekatan utama:
- **Content-Based Filtering**: Merekomendasikan film berdasarkan kemiripan konten (genre, deskripsi, dsb.).
- **Collaborative Filtering**: Merekomendasikan film berdasarkan pola rating pengguna lain yang mirip.

## Data Understanding

Pada tahap ini, dilakukan pemahaman terhadap struktur dan isi dataset yang digunakan. Dataset yang digunakan berasal dari MovieLens dan terdiri dari dua file utama:

- **movies.dat**: Berisi informasi mengenai film, seperti MovieID, judul film, dan genre.
- **ratings.dat**: Berisi data rating yang diberikan oleh pengguna terhadap film, terdiri dari UserID, MovieID, Rating, dan Timestamp.

Data ini akan digunakan untuk membangun sistem rekomendasi dengan dua pendekatan, yaitu content-based filtering dan collaborative filtering.

---

In [3]:
# Data Understanding
import pandas as pd

# Baca file movies.dat
movies = pd.read_csv(
    'datasets/movies.dat',
    sep='::',
    engine='python',
    names=['MovieID', 'Title', 'Genres'],
    encoding='latin1'
)

# Baca file ratings.dat
ratings = pd.read_csv(
    'datasets/ratings.dat',
    sep='::',
    engine='python',
    names=['UserID', 'MovieID', 'Rating', 'Timestamp'],
    encoding='latin1'
)

# Tampilkan beberapa data awal
print("Dataset data film:")
display(movies.head())

print("Dataset data rating:")
display(ratings.head())

Contoh data film:


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Contoh data rating:


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Eksplorasi Data

Eksplorasi data dilakukan untuk mengetahui gambaran umum dataset, seperti jumlah film, jumlah user, jumlah rating, distribusi rating, serta genre yang tersedia. Informasi ini penting untuk memahami karakteristik data sebelum masuk ke tahap persiapan data dan pemodelan.

In [4]:
# Eksplorasi Data

print("Jumlah film unik:", movies['MovieID'].nunique())
print("Jumlah rating:", ratings.shape[0])
print("Jumlah user unik:", ratings['UserID'].nunique())
print("Jumlah genre unik:", len(set('|'.join(movies['Genres']).split('|'))))

print("\nDistribusi rating:")
print(ratings['Rating'].value_counts().sort_index())

print("\nContoh genre yang ada:")
print(set('|'.join(movies['Genres']).split('|')))


Jumlah film unik: 3883
Jumlah rating: 1000209
Jumlah user unik: 6040
Jumlah genre unik: 18

Distribusi rating:
Rating
1     56174
2    107557
3    261197
4    348971
5    226310
Name: count, dtype: int64

Contoh genre yang ada:
{'Action', 'Animation', 'Fantasy', 'Western', 'Documentary', 'Comedy', 'Adventure', 'Romance', 'Drama', 'Sci-Fi', 'Thriller', 'Musical', 'Film-Noir', 'Horror', 'Mystery', 'Crime', 'War', "Children's"}


## Data Preparation

Tahap ini bertujuan untuk membersihkan dan mempersiapkan data sebelum digunakan dalam pemodelan. Langkah-langkah yang dilakukan antara lain:
- Mengecek dan menangani missing value atau duplikasi data.
- Melakukan transformasi data jika diperlukan (misal: ekstraksi fitur genre, encoding, dsb.).
- Menyiapkan data agar siap digunakan untuk content-based filtering maupun collaborative filtering.

In [5]:
# Data Preparation

# Cek missing value pada movies dan ratings
print("Missing value pada movies:\n", movies.isnull().sum())
print("\nMissing value pada ratings:\n", ratings.isnull().sum())

# Cek duplikasi
print("\nDuplikasi pada movies:", movies.duplicated().sum())
print("Duplikasi pada ratings:", ratings.duplicated().sum())

# Pastikan tipe data sesuai
movies['MovieID'] = movies['MovieID'].astype(int)
ratings['UserID'] = ratings['UserID'].astype(int)
ratings['MovieID'] = ratings['MovieID'].astype(int)
ratings['Rating'] = ratings['Rating'].astype(int)

# Ekstrak genre unik untuk keperluan content-based filtering
all_genres = set('|'.join(movies['Genres']).split('|'))
print("\nDaftar genre unik:", all_genres)

Missing value pada movies:
 MovieID    0
Title      0
Genres     0
dtype: int64

Missing value pada ratings:
 UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

Duplikasi pada movies: 0
Duplikasi pada ratings: 0

Daftar genre unik: {'Action', 'Animation', 'Fantasy', 'Western', 'Documentary', 'Comedy', 'Adventure', 'Romance', 'Drama', 'Sci-Fi', 'Thriller', 'Musical', 'Film-Noir', 'Horror', 'Mystery', 'Crime', 'War', "Children's"}


## Modeling

Pada tahap ini, akan dibangun dua model sistem rekomendasi:
1. **Content-Based Filtering**: Merekomendasikan film berdasarkan kemiripan konten (genre).
2. **Collaborative Filtering**: Merekomendasikan film berdasarkan pola rating pengguna lain yang mirip.

Setiap pendekatan akan dijelaskan dan diimplementasikan secara terpisah.

### Content-Based Filtering dengan TF-IDF

Pada pendekatan ini, sistem merekomendasikan film berdasarkan kemiripan konten, yaitu genre film. Genre diubah menjadi representasi numerik menggunakan TF-IDF (Term Frequency-Inverse Document Frequency), lalu dihitung kemiripan antar film menggunakan cosine similarity. Rekomendasi diberikan berdasarkan film yang paling mirip dengan film yang dipilih pengguna.

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Membuat fitur genre sebagai string (tanpa tanda '|')
movies['Genres_str'] = movies['Genres'].apply(lambda x: x.replace('|', ' '))

# Inisialisasi dan fit TF-IDF Vectorizer pada genre
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(movies['Genres_str'])

# Menghitung kemiripan antar film berdasarkan genre (TF-IDF)
cosine_sim_tfidf = cosine_similarity(tfidf_matrix)

# Fungsi rekomendasi film berdasarkan judul (TF-IDF)
def recommend_content_based_tfidf(title, top_n=5):
    idx = movies[movies['Title'].str.lower() == title.lower()].index
    if len(idx) == 0:
        print("Film tidak ditemukan.")
        return
    idx = idx[0]
    sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    film_idx = [i[0] for i in sim_scores]
    return movies.iloc[film_idx][['Title', 'Genres']]

# Contoh rekomendasi
judul_film = 'titanic (1997)'
print(f"Rekomendasi film mirip dengan '{judul_film}':")
display(recommend_content_based_tfidf(judul_film))

Rekomendasi film mirip dengan 'titanic (1997)':


,Title,Genres
24,Leaving Las Vegas (1995),Drama|Romance
34,Carrington (1995),Drama|Romance
45,How to Make an American Quilt (1995),Drama|Romance
48,When Night Is Falling (1995),Drama|Romance
57,"Postino, Il (The Postman) (1994)",Drama|Romance


### Collaborative Filtering (User-Based)

Pada pendekatan ini, sistem merekomendasikan film berdasarkan pola rating pengguna lain yang mirip. Model yang digunakan adalah user-based collaborative filtering dengan cosine similarity. Data rating dibagi menjadi data train dan validasi untuk menghindari data leakage dan memungkinkan evaluasi model.

In [21]:
# Collaborative Filtering: User-Based

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.model_selection import train_test_split

# Membagi data ratings menjadi 80% train dan 20% validasi
ratings_train, ratings_val = train_test_split(
    ratings, test_size=0.2, random_state=42, shuffle=True
)

print("Jumlah data train:", ratings_train.shape[0])
print("Jumlah data validasi:", ratings_val.shape[0])

# Gunakan ratings_train untuk membangun user-item matrix
user_item_matrix = ratings_train.pivot_table(index='UserID', columns='MovieID', values='Rating')
user_item_matrix_filled = user_item_matrix.fillna(0)

# Membuat user-item matrix
user_item_matrix = ratings.pivot_table(index='UserID', columns='MovieID', values='Rating')

# Mengisi missing value dengan 0 (bisa juga pakai mean, tapi 0 untuk baseline)
user_item_matrix_filled = user_item_matrix.fillna(0)

# Hitung similarity antar user
user_sim = cosine_similarity(user_item_matrix_filled)
user_sim_df = pd.DataFrame(user_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

def recommend_collaborative(user_id, top_n=5):
    if user_id not in user_item_matrix.index:
        print("User tidak ditemukan.")
        return
    # Cari user paling mirip (selain dirinya sendiri)
    sim_scores = user_sim_df[user_id].drop(user_id).sort_values(ascending=False)
    top_user = sim_scores.index[0]
    # Cari film yang sudah dirating user mirip, tapi belum dirating user target
    user_movies = set(user_item_matrix.loc[user_id].dropna().index)
    similar_user_movies = set(user_item_matrix.loc[top_user].dropna().index)
    recommend_movies = list(similar_user_movies - user_movies)
    # Ambil top_n film dengan rating tertinggi dari user mirip
    top_movies = user_item_matrix.loc[top_user, recommend_movies].sort_values(ascending=False).head(top_n)
    return movies[movies['MovieID'].isin(top_movies.index)][['Title', 'Genres']]

# Contoh rekomendasi untuk user dengan UserID=1
print("Rekomendasi film untuk UserID=1 (Collaborative Filtering):")
display(recommend_collaborative(1))

Jumlah data train: 800167
Jumlah data validasi: 200042
Rekomendasi film untuk UserID=1 (Collaborative Filtering):


,Title,Genres
898,Some Like It Hot (1959),Comedy|Crime
2009,"Jungle Book, The (1967)",Animation|Children's|Comedy|Musical
2011,Lady and the Tramp (1955),Animation|Children's|Comedy|Musical|Romance
2012,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
3509,Gladiator (2000),Action|Drama
